# Configuration

A MetaDrive instance accepts a dict as the environmental config. For example, you can build a MetaDrive instance with 200 generated maps via 
```python
from metadrive import MetaDriveEnv
config = dict(num_scenarios=200, start_seed=0)
env = MetaDriveEnv(config)
```

In this page, we describe the details of the config system and configurable options for all environments.

## Config system
This section discusses how to configure the an environment in MetaDrive and some features of the config system.

### Overwriting
Every environment has a default config, which records the parameters required to launch the environment. It is content is actually a nested dictionary whose keys and values represent the parameter names and corresponding values. 
This default config dict can be accessed via the class method:
```python
default_config = MetaDriveEnv.default_config()
```

When creating environments, the external config `config` will overwritten default values of certain fields in the `default_config`. The following code exemplifies this. 

In [30]:
from metadrive import MetaDriveEnv
default_config = MetaDriveEnv.default_config()
env = MetaDriveEnv(dict(num_scenarios=100, log_level=50))
env_config = env.config
print("default_config['num_scenarios']:", default_config["num_scenarios"])
print("env_config['num_scenarios']:", env_config["num_scenarios"])

default_config['num_scenarios']: 1
env_config['num_scenarios']: 100


### Sanity Check
There is a check mechanism which prohibit users to set the value for a key that doesn't exist in the `default_config`. This is helpful to make sure that users type the correct parameter name and successfully config the target parameter. 

In [20]:
try:
    env = MetaDriveEnv(dict(non_exist_key=False))
except KeyError as error:
    print(str(error)[:62] + " ...")

"'{'non_exist_key'}' does not exist in existing config. Please ...


The check mechanism will further ensure if the type of the parameter is correct. For example, the `num_scenarios` should be an `int` type, and thus a `list` type parameter will raise an error.   

In [28]:
try:
    env = MetaDriveEnv(dict(num_scenarios=[0, 1]))
except AssertionError as error:
    print(str(error)[:62] + " ...")

TypeError: num_scenarios:[0, 1] ...


### Basic Config Sharing
The default configs are different across all environments, but may share some identical fields. Take the `MetaDriveEnv` and `ScenarioEnv` as example.

In [9]:
from metadrive.envs import MetaDriveEnv, ScenarioEnv
metadrive_config = set(MetaDriveEnv.default_config().keys())
scenario_config = set(ScenarioEnv.default_config().keys())
print("Number of parameters of MetaDriveEnv: {}".format(len(metadrive_config)))
print("Number of parameters of ScenarioEnv: {}\n".format(len(scenario_config)))

try:
    assert metadrive_config == scenario_config
except AssertionError as error:
    print("The config between MetaDriveEnv and ScenarioEnv is different.\n")
    
identical_parameters = scenario_config.intersection(metadrive_config)
print("Number of identical parameters: \
      {}".format(len(identical_parameters)))
print("Number of unique parameters in MetaDriveEnv: \
      {}".format(len(metadrive_config-identical_parameters)))
print("Number of unique parameters in ScenarioEnv: \
      {}".format(len(scenario_config-identical_parameters)))

Number of parameters of MetaDriveEnv: 109
Number of parameters of ScenarioEnv: 121

The config between MetaDriveEnv and ScenarioEnv is different.

Number of identical parameters:       91
Number of unique parameters in MetaDriveEnv:       18
Number of unique parameters in ScenarioEnv:       30


It is worth mentioning the parameter sharing mechanism, which is helpful when we create a new environment, so we don't need to copy some common configs to the `default_config` to the new environments again and again. Let's first check out how the `default_config()` function is implemented.

In [10]:
from metadrive.utils import print_source
print_source(ScenarioEnv.default_config)

@classmethod
def default_config(cls):
    config = super(ScenarioEnv, cls).default_config()
    config.update(SCENARIO_ENV_CONFIG)
    return config



It is quite simple and is implemented by overwriting the `super(ScenarioEnv, cls).default_config()` with `Scenario_ENV_CONFIG`. If we check the contents of the two config dict, we will find that the `BaseEnv.default_config() = super(ScenarioEnv, cls).default_config()` is the subset of `ScenarioEnv.default_config()` and provides the `ScenarioEnv` with the basic configs.

In [22]:
from metadrive.envs.base_env import BaseEnv
set(BaseEnv.default_config()).issubset(set(ScenarioEnv.default_config()))

True

It is the same for the MetaDriveEnv as well, whose default config is:

In [23]:
print_source(MetaDriveEnv.default_config)

@classmethod
def default_config(cls) -> Config:
    config = super(MetaDriveEnv, cls).default_config()
    config.update(METADRIVE_DEFAULT_CONFIG)
    config.register_type("map", str, int)
    config["map_config"].register_type("config", None)
    return config



As there is an overwriting function is called, it is ok to overwrite the values of parameters in `BaseEnv.default_config()` when making the `default_config` for a inherited environment. The following code shows that the config `show_sidewalk` is *True* in `BaseEnv` but is overwritten to *False* in `ScenarioEnv` because of the `SCENARIO_ENV_CONFIG`.

In [25]:
from metadrive.envs.scenario_env import SCENARIO_ENV_CONFIG

assert BaseEnv.default_config()["show_sidewalk"]
assert not ScenarioEnv.default_config()["show_sidewalk"]
assert not SCENARIO_ENV_CONFIG["show_sidewalk"]

## Basic Configs

As all environments are subclass of `BaseEnv` and share the parameters of `BaseEnv`, we first discuss the parameters in `BaseEnv.default_config()`. 
The available items with annotations are listed as follows.
You can check this in the source code as well.

In [1]:
import metadrive.envs.base_env as base_env
from metadrive.utils import print_source, CONFIG
module_source = print_source(base_env, ["BASE_DEFAULT_CONFIG", ")\n\n"], colorscheme=CONFIG)

BASE_DEFAULT_CONFIG = dict(

    # ===== agent =====
    # Whether randomize the car model for the agent, randomly choosing from 4 types of cars
    random_agent_model=False,
    # The ego car config is: env_config["vehicle_config"].update(env_config"[target_vehicle_configs"]["default_agent"])
    target_vehicle_configs={DEFAULT_AGENT: dict(use_special_color=True, spawn_lane_index=None)},

    # ===== multi-agent =====
    # This should be >1 in MARL envs, or set to -1 for spawning as many vehicles as possible.
    num_agents=1,
    # Turn on this to notify the simulator that it is MARL env
    is_multi_agent=False,
    # The number of agent will be fixed adn determined at the start of the episode, if set to False
    allow_respawn=False,
    # How many substepsfor the agent to stay static at the death place after done. env.step = decision_repeat x substeps
    delay_done=0,

    # ===== Action/Control =====
    # Please see Documentation: Action and Policy for more details
    # What 

## Environment Configs
Please see [Environments](rl_environments.ipynb) for unique configs for each environment or check the source code of each environment. 